In [2]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#Libraries for Deep Learning Models
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD
from lightgbm import LGBMClassifier
from tensorflow.keras.utils import to_categorical

#nltk.download('vader_lexicon')
#load all the wsb_* files
stocks = ["GME", "AMC", "BB", "TSLA", "PLTR", "MSFT", "BBBY", "SPY", "AAPL"]

In [3]:
#from google.colab import drive
#drive.mount('/content/drive/')
#dirpath = '/content/drive/MyDrive/FT5005/GW_dataset/'
dirpath = ''

Mounted at /content/drive/


In [4]:
# load the data
# Here I just use AAPL as test
stock = "AAPL"
df_stock_price = pd.read_csv(dirpath + f"yahoo stock data/{stock}.csv")
df_collated = pd.read_csv(dirpath + f"Collated/collated_{stock}.csv")

#df_stock_price = pd.read_csv(f"{stock}.csv")
#df_collated = pd.read_csv(f"collated_{stock}.csv")

In [5]:
# Convert 'date' feature into date type
df_stock_price['Date'] = pd.to_datetime(df_stock_price['date'], format='%Y-%m-%d').dt.date
df_sortedbyDate_stock_price = df_stock_price.sort_values(by='Date').dropna(subset=['Date'], inplace=False)


In [6]:
# Get the price of t-lag_t
# for example, if the date_t-1 is not working day, then we'll find the latest work date as date_t-1
# if date_t-1 and date_t-2 are not working days, date_t-1 will be the latest work date, t-2 will be one day ahead of date_t-1
import datetime
def get_previous_price(date, df_stock_price, lag_t):
  count = lag_t
  lag_date = date
  while count > 0:
    lag_date = lag_date - datetime.timedelta(days=1)
    if lag_date not in df_stock_price['Date'].values:
      continue
    else:
      count -= 1

  return df_stock_price.loc[df_stock_price['Date'] == lag_date]['close'].values[0]

In [7]:
# get the columns we need
df_collated_dataset = df_collated[['date','volume', 'labels_shifted', 'score', 'sentiment_score','capital_letter_ratio','profanity_count','emoji_count','capital_letter_count','post_length']]

df_collated_dataset['date2'] = pd.to_datetime(df_collated_dataset['date'], format='%Y-%m-%d').dt.date

<ipython-input-7-435979c76a37>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_collated_dataset['date2'] = pd.to_datetime(df_collated_dataset['date'], format='%Y-%m-%d').dt.date


In [8]:
# Get historical price
df_collated_dataset['price_t-1'] = df_collated_dataset['date2'].apply(lambda x: get_previous_price(x, df_sortedbyDate_stock_price,1))
df_collated_dataset['price_t-2'] = df_collated_dataset['date2'].apply(lambda x: get_previous_price(x, df_sortedbyDate_stock_price,2))

<ipython-input-8-fd9295c84810>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_collated_dataset['price_t-1'] = df_collated_dataset['date2'].apply(lambda x: get_previous_price(x, df_sortedbyDate_stock_price,1))
<ipython-input-8-fd9295c84810>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_collated_dataset['price_t-2'] = df_collated_dataset['date2'].apply(lambda x: get_previous_price(x, df_sortedbyDate_stock_price,2))


In [9]:
df_collated_dataset

,date,volume,labels_shifted,score,sentiment_score,capital_letter_ratio,profanity_count,emoji_count,capital_letter_count,post_length,date2,price_t-1,price_t-2
0,2021-01-29,177523800,1.0,498.571429,-0.243786,0.169031,43.0,37.0,879.0,12875.0,2021-01-29,137.089996,142.059998
1,2021-02-01,106239800,1.0,207.250000,0.270533,0.046063,62.0,354.0,4029.0,76115.0,2021-02-01,131.960007,137.089996
2,2021-02-02,83305400,0.0,846.454545,0.337473,0.035424,29.0,40.0,2528.0,69182.0,2021-02-02,134.139999,131.960007
3,2021-02-03,89880900,1.0,114.600000,0.576980,0.064974,27.0,16.0,1291.0,14847.0,2021-02-03,134.990005,134.139999
4,2021-02-04,84183100,0.0,6172.600000,0.640170,0.036137,21.0,3.0,1000.0,26299.0,2021-02-04,133.940002,134.990005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,2023-03-09,53833600,0.0,3.000000,0.146967,0.052511,0.0,1.0,14.0,279.0,2023-03-09,152.869995,151.600006
531,2023-03-10,68572400,1.0,6.000000,0.699800,0.091118,0.0,0.0,88.0,1124.0,2023-03-10,150.589996,152.869995
532,2023-03-13,84457100,1.0,3.500000,-0.210750,0.057266,0.0,0.0,12.0,214.0,2023-03-13,148.500000,150.589996
533,2023-03-14,73695900,1.0,5.000000,0.000000,0.050505,0.0,1.0,5.0,99.0,2023-03-14,150.470001,148.500000


In [33]:
# get price difference
df_collated_dataset['dif_price'] = df_collated['close']-df_collated['open']

<ipython-input-33-04a66ac04036>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_collated_dataset['dif_price'] = df_collated['close']-df_collated['open']


In [34]:
HP = ['price_t-1', 'price_t-2']
HP_V = ['price_t-1', 'price_t-2','volume']
HP_V_TF = ['price_t-1', 'price_t-2','volume','sentiment_score','capital_letter_ratio','profanity_count','emoji_count','capital_letter_count','post_length']
All_features = ['dif_price','volume', 'score', 'sentiment_score','capital_letter_ratio','profanity_count','emoji_count','capital_letter_count','post_length','price_t-1','price_t-2']
X = df_collated_dataset[All_features]
Y = df_collated_dataset['labels_shifted']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=False)


In [28]:
# Here I try with the base input features(price_t-1, price_t-2, sentiment_score).

from time import time
time1 = time()
param_grid = {
    'n_estimators': range(400,500,20),
    'max_depth': range(3,6),
    'learning_rate':[0.003, 0.005, 0.007, 0.1,0.2 ,0.3]
}

# I try different input features, the simplest is price_t-1, price_t-2 and sentiment_score, which performs the best yet
X = df_collated_dataset[['score', 'volume','sentiment_score','post_length','price_t-1', 'price_t-2']]
Y = df_collated_dataset['labels_shifted']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=False)

# Perform grid search with 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=False)
grid_search = GridSearchCV(LGBMClassifier(random_state=42), param_grid, cv=kf, scoring='accuracy',verbose=1)
grid_search.fit(X_train, y_train)


print('Report the best parameters and rmse for best result:')
print('##############################################################################################')
print('Grid Search Best Parameters: ', grid_search.best_params_)
print('Grid Search Best accuracy: ', grid_search.best_score_)
print('##############################################################################################')

best_grid_model = grid_search.best_estimator_

y_pred = best_grid_model.predict(X_test)

# Calculate the RMSE for each model
pre_accuracy_score = accuracy_score(y_test, y_pred)

print("Predicted accuracy score:", pre_accuracy_score)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Report the best parameters and rmse for best result:
##############################################################################################
Grid Search Best Parameters:  {'learning_rate': 0.005, 'max_depth': 4, 'n_estimators': 420}
Grid Search Best accuracy:  0.5330506155950753
##############################################################################################
Predicted accuracy score: 0.5327102803738317


In [45]:
# also here, I tried other methods, and lightGMB has the best performance(I'm still trying to tuning other models as well.)
models = []
models.append(('LR', LogisticRegression(n_jobs=-1)))
# models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))

#Neural Network"
models.append(('NN', MLPClassifier(hidden_layer_sizes=(5,3),activation='relu',solver='sgd'))) # replace by NN with 2 hidden layer, 5 neurons, 3 neurons each, relu activation, SGD optimizer. 

models.append(('SVM',SVC()))
models.append(('LGBM', LGBMClassifier(learning_rate=0.005, max_depth=4, n_estimators = 420)))


# Bagging methods
models.append(('RF', RandomForestClassifier()))

In [43]:
from sklearn.model_selection import TimeSeriesSplit
def kfold_training(name, model, X, Y):
    model.fit(X,Y)
    tscv = TimeSeriesSplit(n_splits=5)
    cv_results = cross_val_score(model, X, Y, cv=tscv, scoring='accuracy')
    y_pred = model.predict(X_test)
    pre_accuracy_score = accuracy_score(y_test, y_pred)
    print(name, "predicted accuracy score:", pre_accuracy_score)
    return cv_results

In [46]:
results = []
names = []
for name, model in models:
    cv_results = kfold_training(name, model, X_train, y_train)
    print("accuracy of ", name, "is", cv_results)
    results.append(cv_results.mean())
    names.append(name)

LR predicted accuracy score: 0.5327102803738317
accuracy of  LR is [0.53521127 0.57746479 0.43661972 0.50704225 0.49295775]
KNN predicted accuracy score: 0.5607476635514018
accuracy of  KNN is [0.52112676 0.54929577 0.6056338  0.52112676 0.47887324]
CART predicted accuracy score: 0.4205607476635514
accuracy of  CART is [0.42253521 0.50704225 0.46478873 0.43661972 0.46478873]
NB predicted accuracy score: 0.4485981308411215
accuracy of  NB is [0.46478873 0.4084507  0.54929577 0.49295775 0.52112676]
NN predicted accuracy score: 0.4672897196261682
accuracy of  NN is [0.46478873 0.57746479 0.43661972 0.50704225 0.49295775]
SVM predicted accuracy score: 0.5327102803738317
accuracy of  SVM is [0.46478873 0.57746479 0.43661972 0.50704225 0.49295775]
LGBM predicted accuracy score: 0.5700934579439252
accuracy of  LGBM is [0.36619718 0.4084507  0.46478873 0.54929577 0.43661972]
RF predicted accuracy score: 0.5233644859813084
accuracy of  RF is [0.47887324 0.53521127 0.49295775 0.52112676 0.591549